In [1]:
import sys, os
sys.path.insert(0, '/home/llr/cms/wind/cmssw/CMSSW_9_4_2/src/ZZAnalysis/AnalysisStep/test/Python/')

In [2]:
from trainlib.ConfigFileHandler import ConfigFileHandler
from trainlib.ModelCollectionConfigFileHandler import ModelCollectionConfigFileHandler

Welcome to JupyROOT 6.10/09


Using TensorFlow backend.


In [3]:
from vis.losses import ActivationMaximization

In [4]:
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

In [5]:
K.set_learning_phase(False)

In [6]:
def get_model(run_dir, model_name):
    confhandler = ModelCollectionConfigFileHandler()
    confhandler.load_configuration(os.path.join(run_dir, "settings.conf"))
    
    mcolls = confhandler.GetModelCollection(weightpath = os.path.join(run_dir, "training/"))
    
    for mcoll in mcolls:
        if model_name in mcoll.model_dict.keys():
            found_mcoll = mcoll
            model = mcoll.model_dict[model_name]
            pre = mcoll.preprocessor_dict[model_name]
            break
        
    return model, pre

In [65]:
# load the neural network that is to be visualized
run_dir = "/data_CMS/cms/wind/180508_bkg_cat_ZXonly/number_layers_2.0_number_neurons_64.0/"
model_name = "D_VBF_ggH_2j_ML"

In [66]:
model, pre = get_model(run_dir, model_name)

attempting to load configuration file from /data_CMS/cms/wind/180508_bkg_cat_ZXonly/number_layers_2.0_number_neurons_64.0/settings.conf
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 28 inputs
FlexiblePCAPreprocessor set up for 28 inputs
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 10 inputs
FlexiblePCAPreprocessor set up for 10 inputs
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 15 inputs
FlexiblePCAPreprocessor set up for 15 inputs
found the following models belonging to this collection:
D_VBF_ggH_2j_ML
D_VBF_ggH_0j_ML
D_VBF_ggH_1j_ML
now attempting to load model from file /data_CMS/cms/wind/180508_bkg_cat_ZXonly/number_layers_2.0_number_neurons_64.0/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/final.hdf5
read weights from /data_CMS/cms/wind/180508_bkg_cat_ZXonly/number_layers_2.0_number_neurons_64.0/

In [67]:
pre.processed_columns

Index([       u'JetPt_2',          u'PFMET', u'D_VBF2j_ZHh_ME',
       u'D_VBF2j_WHh_ME', u'D_VBF2j_ggH_ME',        u'JetPt_0',
               u'Z2Mass',       u'JetEta_1',           u'Z1Pt',
            u'ZZMassErr',        u'JetPt_1',   u'D_WHh_ggH_ME',
         u'D_ZHh_ggH_ME',         u'Z1Mass',           u'Z2Pt',
             u'JetEta_2',           u'ZZPt',   u'D_WHh_ZHh_ME',
                u'ZZEta',       u'JetEta_0',   u'JetPhi_2_sin',
         u'JetPhi_2_cos',   u'JetPhi_0_sin',   u'JetPhi_0_cos',
            u'ZZPhi_sin',      u'ZZPhi_cos',   u'JetPhi_1_sin',
         u'JetPhi_1_cos'],
      dtype='object')

In [68]:
keras_model = model.get_keras_model()

In [69]:
layer_dict = {layer.name: layer for layer in keras_model.layers}

In [70]:
layer_dict

{u'D_VBF_ggH_2j_ML_input': <keras.engine.topology.InputLayer at 0x7f9106b5ff50>,
 'activation_142': <keras.layers.core.Activation at 0x7f9106b5fd50>,
 'activation_143': <keras.layers.core.Activation at 0x7f91069e0410>,
 'activation_144': <keras.layers.core.Activation at 0x7f910686cbd0>,
 'batch_normalization_142': <keras.layers.normalization.BatchNormalization at 0x7f9106bb5e10>,
 'batch_normalization_143': <keras.layers.normalization.BatchNormalization at 0x7f91069c7090>,
 'batch_normalization_144': <keras.layers.normalization.BatchNormalization at 0x7f9106c23910>,
 'dense_142': <keras.layers.core.Dense at 0x7f9106dfdc10>,
 'dense_143': <keras.layers.core.Dense at 0x7f9106d7b890>,
 'dense_144': <keras.layers.core.Dense at 0x7f9106910050>,
 'dropout_142': <keras.layers.core.Dropout at 0x7f9106a99b90>,
 'dropout_143': <keras.layers.core.Dropout at 0x7f91069105d0>,
 'dropout_144': <keras.layers.core.Dropout at 0x7f910c76ed10>,
 'target': <keras.layers.core.Dense at 0x7f9106d578d0>}

In [71]:
def normalize(x):
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [72]:
layer_output = layer_dict["target"].output

In [73]:
inputs = keras_model.input

In [74]:
loss = K.mean(layer_output[0])

In [75]:
grads = K.gradients(loss, inputs)[0]
grads = normalize(grads)

In [76]:
iterate = K.function([inputs], [loss, grads])

In [77]:
stepsize = 0.1

In [78]:
def synthesize_input_from(cur_input, threshold):
    for i in range(200):
        cur_loss, cur_grads = iterate([cur_input])
        cur_input += stepsize * cur_grads

        if cur_loss > threshold:
            break
            
    pre_pca_inputs = pre.base_preprocessor.pca.inverse_transform(cur_input).flatten().tolist()
    synthesized_input = {key: val for (key, val) in zip(pre.processed_columns, pre_pca_inputs)}
    
    return synthesized_input

In [79]:
def synthesize_inputs(num_inputs, threshold):
    synthesized = []
    
    for i in range(num_inputs):
        # start from a random input
        cur_input = np.random.uniform(-1, 1, size = (1, keras_model.input_shape[1]))
        synthesized_input = synthesize_input_from(cur_input, threshold)
        
        synthesized.append(synthesized_input)
        
    return synthesized

In [80]:
def extract_column(input_dicts, colname):
    retval = [cur_dict[colname] for cur_dict in input_dicts]
    
    return retval

In [96]:
def make_activation_maximization_plot(x_column, y_column, x_label, y_label, x_range, y_range, threshold):
    synthesized = synthesize_inputs(100000, threshold = threshold)
    
    x_data = extract_column(synthesized, x_column)
    y_data = extract_column(synthesized, y_column)
    
    x_edges = np.linspace(*x_range)
    y_edges = np.linspace(*y_range)
    
    H, x_edges, y_edges = np.histogram2d(x_data, y_data, bins = (x_edges, y_edges))
    
    fig = plt.figure(figsize = (5, 5))
    ax = fig.add_subplot(111)

    cax = ax.imshow(H, interpolation = "nearest", cmap = "hot", origin = "low", 
                    extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
    
    ax.set_xlabel(x_label, fontsize = 17)
    ax.set_ylabel(y_label, fontsize = 17)

    #fig.colorbar(cax)
    plt.tight_layout()

    return fig

In [97]:
outpath = "/data_CMS/cms/wind/ActivationMaximization/"

In [98]:
fig = make_activation_maximization_plot("D_VBF2j_ggH_ME", "D_VBF2j_ZHh_ME", 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ggH}}^{\mathrm{ME}}$', 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ZH-hadr}}^{\mathrm{ME}}$', 
                                        (0, 1, 40), (0, 1, 40), threshold = 0.3)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_MELA_03.pdf"))

In [99]:
fig = make_activation_maximization_plot("D_VBF2j_ggH_ME", "D_VBF2j_ZHh_ME", 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ggH}}^{\mathrm{ME}}$', 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ZH-hadr}}^{\mathrm{ME}}$', 
                                        (0, 1, 40), (0, 1, 40), threshold = 0.5)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_MELA_05.pdf"))

In [100]:
fig = make_activation_maximization_plot("D_VBF2j_ggH_ME", "D_VBF2j_ZHh_ME", 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ggH}}^{\mathrm{ME}}$', 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ZH-hadr}}^{\mathrm{ME}}$', 
                                        (0, 1, 40), (0, 1, 40), threshold = 0.7)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_MELA_07.pdf"))

In [101]:
fig = make_activation_maximization_plot("D_VBF2j_ggH_ME", "D_VBF2j_ZHh_ME", 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ggH}}^{\mathrm{ME}}$', 
                                        r'$\mathcal{D}_{\mathrm{VBF-2j, ZH-hadr}}^{\mathrm{ME}}$', 
                                        (0, 1, 40), (0, 1, 40), threshold = 0.9)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_MELA_09.pdf"))

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-matplotlib/1.5.2-fmblme/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [85]:
fig = make_activation_maximization_plot("JetEta_0", "JetEta_1", r'$\eta_{\,\,0}$', r'$\eta_{\,\,1}$', (-6, 6, 40), (-6, 6, 40), threshold = 1.0 - 1e-2)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_099.pdf"))

In [86]:
fig = make_activation_maximization_plot("JetEta_0", "JetEta_1", r'$\eta_{\,\,0}$', r'$\eta_{\,\,1}$', (-6, 6, 40), (-6, 6, 40), threshold = 1.0 - 1e-3)
plt.savefig(os.path.join(outpath, "D_VBF_ggH_0999.pdf"))